### ❤ 인스타그램 크롤러 💬
* 실행하기 전 본인의 크롬 버전에 맞게 [chromedriver을 설치](https://chancoding.tistory.com/136)해주세요. 
* shift+enter 혹은 상단 메뉴바 Run을 누르면 각 블록을 실행할 수 있습니다.
* 실행 후 2단계 보안이 해제된 계정으로 로그인 하신 후 원하는 해시태그를 입력합니다.
* 스크롤 시간에 따라 게시물 크롤하는 피드의 수가 달라집니다. 

#### 에러 사항
* Http error 500 발생 원인은 과도한 서버 호출로 인한 오류이므로 해당 링크를 다시 호출하면 해결됩니다.

In [62]:
#라이브러리 정의 및 불러오기
from selenium import webdriver
import urllib.parse
from urllib.request import Request, urlopen
import time
import datetime
import pandas as pd
from bs4 import BeautifulSoup
from urllib.error import HTTPError

In [63]:
# 스크롤다운 함수
def doScrollDown(whileSeconds):
    start = datetime.datetime.now()
    end = start + datetime.timedelta(seconds=whileSeconds)
    
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(1)
        if datetime.datetime.now() > end:
            break

In [64]:
# 피드 링크 가져오기

def get_feeds(driver):
    feed_list = driver.find_elements_by_tag_name("div")
    feed_links = []
    for feed in feed_list:
        class_name = feed.get_attribute("class")
        if class_name.startswith("v1Nh3"):
            link = feed.find_element_by_tag_name("a").get_attribute("href")
            feed_links.append(link)
    return feed_links

In [65]:
# 해시태그 가져오기
def get_hashtags(driver):
    tags = []
    data = driver.find_elements_by_tag_name("a")
    for datum in data:
        href_val = datum.get_attribute("href")
        if "/explore/tags/" in href_val:
            tags.append(datum.text.replace("#",""))

    return tags

In [66]:
# 태그된 계정 가져오기
def get_taggedaccs(driver):
    acc_list=[]
    upload_accs = driver.find_elements_by_tag_name("a")
    try:
        for acc in upload_accs:
            class_name = acc.get_attribute("class")
            if class_name == "notranslate":
                acc_list.append(acc.text.replace("@",""))
    except:
        pass
    return acc_list

In [71]:
# 피드 내용 가져오기

def get_feed_content(driver, info):
    # post content
    post_content=driver.find_element_by_class_name("C4VMK")
    info["post_content"] = post_content.text
    
    # hashtags and taagged accounts
    hashtags = get_hashtags(driver)
    tagaccs = get_taggedaccs(driver)
    info["hashtags"] = hashtags
    info["taggedaccs"] = tagaccs

    
    # post likes
    try:
        likes = driver.find_element_by_class_name("Nm9Fw")
        info["likes"] = likes.find_element_by_tag_name("span").text
        info["views"] = "unknown"
    except:
        # 좋아요 외 영상 조회수 확인
        try:
            views = driver.find_element_by_class_name("vcOH2")
            info["views"] = views.find_element_by_tag_name("span").text
            info["likes"] = "unknown"
        except:
            pass
    
    # post date
    date = driver.find_element_by_tag_name("time")
    info["date"] = date.get_attribute("datetime")
        
    return info["date"]

### 알고리즘 시작

In [68]:
# 크롬 드라이버 불러오기
chromedriver="./chromedriver_win32/chromedriver.exe"
driver=webdriver.Chrome(chromedriver)

In [69]:
## Selenium 속도 향상 (옵션)
options = webdriver.ChromeOptions() # 크롬 옵션 객체 생성
options.add_argument('headless') # headless 모드 설정
options.add_argument("window-size=1920x1080") # 화면크기(전체화면)
options.add_argument("disable-gpu") 
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")

# 속도 향상을 위한 옵션 해제
prefs = {'profile.default_content_setting_values': {'cookies' : 2, 
                                                    'images': 2, 
                                                    'plugins' : 2, 
                                                    'popups': 2, 
                                                    'geolocation': 2, 
                                                    'notifications' : 2, 
                                                    'auto_select_certificate': 2, 
                                                    'fullscreen' : 2, 
                                                    'mouselock' : 2, 
                                                    'mixed_script': 2,
                                                    'media_stream' : 2, 
                                                    'media_stream_mic' : 2, 
                                                    'media_stream_camera': 2, 
                                                    'protocol_handlers' : 2, 
                                                    'ppapi_broker' : 2, 
                                                    'automatic_downloads': 2, 
                                                    'midi_sysex' : 2, 
                                                    'push_messaging' : 2, 
                                                    'ssl_cert_decisions': 2, 
                                                    'metro_switch_to_desktop' : 2, 
                                                    'protected_media_identifier': 2, 
                                                    'app_banner': 2, 
                                                    'site_engagement' : 2, 
                                                    'durable_storage' : 2}}   
options.add_experimental_option('prefs', prefs)

In [70]:
# login
loginUrl = 'https://www.instagram.com/accounts/login/'
driver.get(loginUrl)
time.sleep(2)

# 인스타그램 계정 설정 > 보안 > 2단계 설정 미리 해제할 것
#username = input("유저 아이디 입력: ")
username="99oop8ujj2"
#userpw = input("유저 비밀번호 입력: ")
userpw = "q0070p0070"

driver.find_element_by_name('username').send_keys(username)
driver.find_element_by_name('password').send_keys(userpw)
time.sleep(2)
driver.find_element_by_css_selector('button.sqdOP.L3NKy.y3zKF').click()
time.sleep(20)

KeyboardInterrupt: 

In [ ]:
# 정보 나중에 저장하기 클릭하고 넘어가기
try:
    driver.find_element_by_css_selector('button.sqdOP.yWX7d.y3zKF').click()
    time.sleep(3)
except:
    pass
# 설정 나중에하기 클릭하고 넘어가기
try:
    driver.find_element_by_css_selector('button.aOOlW.HoLwm').click()
    time.sleep(3)
except:
    pass

In [ ]:
keyword=input("해시태그를 입력하세요: ")
encText = urllib.parse.quote(keyword)

In [ ]:
url="https://www.instagram.com/explore/tags/"+str(encText)+'/'
driver.get(url)
time.sleep(3)

In [ ]:
scroll_time = int(input("스크롤 시간 입력: "))
doScrollDown(scroll_time)

In [ ]:
feed_links = get_feeds(driver)
#print("검색된 피드 갯수: "+str(len(feed_links)))

In [ ]:
# 검색결과 창에 있는 피드 정보 수집
all_accounts = set() # 계정 수집
all_posts_info = {}
post_index = 0
for link in feed_links:
    post_link = link
    driver.get(str(post_link))
    time.sleep(3)
    info = {}
    info["link"] = post_link
    
    # account name
    account=driver.find_elements_by_tag_name("a")
    for acc in account:
        if acc.get_attribute("class").startswith("sqdOP"):
            info["account"] = acc.text
            break

    all_accounts.add(info["account"])

    get_feed_content(driver, info)
    
    all_posts_info[post_index] = info
    post_index +=1

In [58]:
print(all_accounts)

{'ondo_case', 'j.xh___', 'always_with_color', 'graycase_e', 'lilzyumiko', 'dooxoeazbid', 'my_youth_of_today', 'bafgf12', 'ibidkrhyun6732', 'heeje0ng_e', 'neoul_leather', 'official_ohimall', 'ohi_market', 'aumincgapp5919', 'eiwonjckim1364', 'jthefloor_', 'suruging.v', 'fam_goods1', 'banloo_sfly93', 'iocharmmkmy6055', 'mejwi_mbig33', '14banazijbit', 'wuwhitezvlike5041', 'verotio.official', 'highnfarm3', 'eleftgnsly6601', 'won_did2426', 'daycover_', '_jeong__han_', '91.hellene', 'hyun__940', 'jj_vely._', 'w3o_r1o', 'madeyoon.kr', 'utenqcbig4937'}


In [59]:
# 정보에 있는 계정 수집

all_acc_info = []
for acc in all_accounts:
    info = {}

    link = "https://instagram.com/"+acc+"/"
     #account info
    #try: 
        #html = urlopen(link)
    #except HTTPError as e:
        #print(e)
    if link == "":
        pass
    else:
        driver.get(link)
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
        time.sleep(3)

        acc_info = driver.find_elements_by_class_name("g47SY ")
        acc_info_list = []
        for num in acc_info:
            acc_info_list.append(num)
        info["account"] = acc
        info["account_posts"] = acc_info_list[0].text
        info["account_followers"] = acc_info_list[1].text
        info["account_following"] = acc_info_list[2].text
        info["account_intro"] = driver.find_element_by_class_name("-vDIg").text

        feed_links = get_feeds(driver)
        account_feeds = []
        for feed in feed_links:
            driver.get(feed)
            time.sleep(2)

            feed_content = {}
            try:
                date = get_feed_content(driver, feed_content)
                account_feeds.append(feed_content)
            except:
                driver.get(feed)
                time.sleep(2)
                
                date = get_feed_content(driver, feed_content)
                account_feeds.append(feed_content)

        info["account_feeds"] = account_feeds
        all_acc_info.append(info)
        #print(all_acc_info)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".C4VMK"}
  (Session info: chrome=92.0.4515.131)


In [60]:
# 마지막에 엑셀로 정리하기

df = pd.json_normalize(all_acc_info, record_path=["account_feeds"],
                      meta=['account', 
                            "account_posts", 
                            "account_followers",
                            "account_following", 
                            "account_intro"])
df = df.reindex(columns=['account', 
                         'account_posts',
                         'account_followers',
                         "account_following",
                         "account_intro",
                         "post_content",
                         "hashtags",
                         "taggedaccs",
                         "likes",
                         "views",
                         "date"])
df

,account,account_posts,account_followers,account_following,account_intro,post_content,hashtags,taggedaccs,likes,views,date
0,ondo_case,475,21.6천,244,온도샵(ONDOSHOP)\n웹사이트\n#온도샵 #케이스쇼핑몰 #아이폰케이스...,ondo_case\n투명 젤리에 카드 수납까지 ♡\n온도샵에서 최저가 판매중입니다 ...,[],[],217,unknown,2021-08-08T09:20:01.000Z
1,ondo_case,475,21.6천,244,온도샵(ONDOSHOP)\n웹사이트\n#온도샵 #케이스쇼핑몰 #아이폰케이스...,ondo_case\n여름이랑 잘 어울리는 라이트퍼플 색상이예요 💟\n비오는 주말이지...,[],[],328,unknown,2021-08-08T08:39:45.000Z
2,ondo_case,475,21.6천,244,온도샵(ONDOSHOP)\n웹사이트\n#온도샵 #케이스쇼핑몰 #아이폰케이스...,ondo_case\n오늘이 실리콘 케이스 9900 𝗟𝗮𝘀𝘁 𝗗𝗮𝘆\n랜덤팩 증정 5...,[],[],365,unknown,2021-08-08T02:26:47.000Z
3,ondo_case,475,21.6천,244,온도샵(ONDOSHOP)\n웹사이트\n#온도샵 #케이스쇼핑몰 #아이폰케이스...,ondo_case\n오늘 구매시 실리콘 9900원 ♡♡\n\n글라스 포함이랑 애플워...,[],[],376,unknown,2021-08-07T10:07:26.000Z
4,ondo_case,475,21.6천,244,온도샵(ONDOSHOP)\n웹사이트\n#온도샵 #케이스쇼핑몰 #아이폰케이스...,ondo_case\n청량한 느낌의 피스타치오 🍃\n이번주도 진짜 다들 믿고\n구매해...,"[온도케이스, 색감맛집, 아이폰케이스]",[],367,unknown,2021-08-07T02:37:23.000Z
...,...,...,...,...,...,...,...,...,...,...,...
315,ohi_market,254,"3,702",161,애플워치스트랩 + 줄질 + 오하이마켓🌈\n쇼핑 및 유통\n#오하이마켓\n말...,"ohi_market\n✨오하이마켓 체인 메탈 스트랩✨\n시장 최저가 8,900원!!...","[애플워치, 줄질, 메탈시계, 여성메탈시계, 애플워치줄질, 애플워치메탈스트랩, 체인...",[],26,unknown,2021-08-02T03:44:10.000Z
316,ohi_market,254,"3,702",161,애플워치스트랩 + 줄질 + 오하이마켓🌈\n쇼핑 및 유통\n#오하이마켓\n말...,"ohi_market\n🌊애플워치 투명 버클형 스트랩🌊\n시장 최저가 단돈 7,900...",[],[],83,unknown,2021-08-01T08:20:59.000Z
317,ohi_market,254,"3,702",161,애플워치스트랩 + 줄질 + 오하이마켓🌈\n쇼핑 및 유통\n#오하이마켓\n말...,ohi_market\n독보적 인기템 클래식 메탈 스트랩 🖤\n어떤 룩에나 찰떡인 클...,"[애플워치, 줄질, 애플워치스트랩, 메탈스트랩, 가죽스트랩, 애플워치밴드, 애플워치...",[],15,unknown,2021-08-01T05:44:35.000Z
318,ohi_market,254,"3,702",161,애플워치스트랩 + 줄질 + 오하이마켓🌈\n쇼핑 및 유통\n#오하이마켓\n말...,ohi_market\n🌊메탈 스트랩 최대50% 할인 이벤트🌊\n1만원대 가격으로 줄...,[],"[when_i_say_han, beannfairy]",214,unknown,2021-08-01T03:01:24.000Z


In [61]:
filename = str(datetime.datetime.now())[:12]
df.to_csv("{}.csv".format(filename),index=False, encoding="utf-8-sig")